In [1]:
from datasets import load_dataset, Dataset, DatasetDict
import re

In [2]:
pal_demos = [
  {
    "question": "Olivia has $23. She bought five bagels for $3 each. How much money does she have left?",
    "reasoning": "She bought 5 bagels for $3 each. This means she spent $15. She has $8 left.",
    "answer": "8"
  },
  {
    "question": "Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?",
    "reasoning": "Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has <<58 - 23 = 35>>35 balls.\nOn Wednesday he lost 2 more so now he has <<35 - 2 = 33>>33 balls.",
    "answer": "33"
  },
  {
    "question": "There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?",
    "reasoning": "There are 4 days from monday to thursday. 5 computers were added each day. That means in total <<4 * 5 = 20>>20 computers were added.\nThere were 9 computers in the beginning, so now there are <<9 + 20 = 29>>29 computers.",
    "answer": "29"
  },
  {
    "question": "Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?",
    "reasoning": "He has 5 toys. He got 2 from mom, so after that he has <<5 + 2 = 7>>7 toys.\nThen he got 2 more from dad, so in total he has <<7 + 2 = 9>>9 toys.",
    "answer": "9"
  },
  {
    "question": "Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?",
    "reasoning": "Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of lollipops he has given to Denny must have been <<20 - 12 = 8>>8 lollipops.",
    "answer": "8"
  },
  {
    "question": "Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?",
    "reasoning": "Leah had 32 chocolates and Leah's sister had 42. That means there were originally <<32 + 42 = 74>>74 chocolates.\n35 have been eaten. So in total they still have <<74 - 35 = 39>>39 chocolates.",
    "answer": "39"
  },
  {
    "question": "If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?",
    "reasoning": "There are 3 cars in the parking lot already. 2 more arrive. Now there are <<3 + 2 = 5>>5 cars.",
    "answer": "5"
  },
  {
    "question": "There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?",
    "reasoning": "We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted. So, they must have planted <<21 - 15 = 6>>6 trees.",
    "answer": "6"
  }
]


In [3]:
sc_demos = [
    {
    "question": "There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?",
    "reasoning": "We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted. So, they must have planted <<21 - 15 = 6>>6 trees.",
    "answer": "6"
  },
  {
    "question": "If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?",
    "reasoning": "There are 3 cars in the parking lot already. 2 more arrive. Now there are <<3 + 2 = 5>>5 cars.",
    "answer": "5"
  },
  {
    "question": "Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?",
    "reasoning": "Leah had 32 chocolates and Leah's sister had 42. That means there were originally <<32 + 42 = 74>>74 chocolates.\n35 have been eaten. So in total they still have <<74 - 35 = 39>>39 chocolates.",
    "answer": "39"
  },
  {
    "question": "Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?",
    "reasoning": "Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of lollipops he has given to Denny must have been <<20 - 12 = 8>>8 lollipops.",
    "answer": "8"
  },
  {
    "question": "Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?",
    "reasoning": "He has 5 toys. He got 2 from mom, so after that he has <<5 + 2 = 7>>7 toys.\nThen he got 2 more from dad, so in total he has <<7 + 2 = 9>>9 toys.",
    "answer": "9"
  },
  {
    "question": "There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?",
    "reasoning": "There are 4 days from monday to thursday. 5 computers were added each day. That means in total <<4 * 5 = 20>>20 computers were added.\nThere were 9 computers in the beginning, so now there are <<9 + 20 = 29>>29 computers.",
    "answer": "29"
  },
  {
    "question": "Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?",
    "reasoning": "Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has <<58 - 23 = 35>>35 balls.\nOn Wednesday he lost 2 more so now he has <<35 - 2 = 33>>33 balls.",
    "answer": "33"
  },
  {
    "question": "Olivia has $23. She bought five bagels for $3 each. How much money does she have left?",
    "reasoning": "She bought 5 bagels for $3 each. This means she spent $15. She has $8 left.",
    "answer": "8"
  },
]


In [4]:
def react_trajectory(row):
    question = row["question"]
    answer = row["answer"]
    reasoning = row["reasoning"].splitlines()
    trajectory = [{"question": question.strip()}]
    res = answer

    for line in reasoning:
        pattern = (
            r"(?P<pre>(=(\ )?|equals(\ )?)?(\$)?)<<(?P<exp>.*?)=(?P<res>.*?)>>([^\s]*)"
        )
        expressions = re.search(pattern, line)

        if expressions is None:
            trajectory += [
                {"thought": line.strip().replace("  ", " ")},
            ]
        else:
            thought = re.sub(pattern, "", line)
            thought = thought.rstrip(".").rstrip(",")
            exp = expressions.group("exp")
            res = expressions.group("res")

            trajectory += [
                {
                    "thought": f"{thought.strip().replace('  ', ' ')}. I need to calculate {exp.strip()}"
                },
                {"action": f"Calculator[{exp.strip()}]"},
                {"observation": res.strip()},
            ]
    if next(iter(trajectory[-1].keys())) == "observation":
        trajectory.append({"thought": f"The answer is {answer}"})

    trajectory.append({"action": f"Finish[{answer}]"})

    traj_keys = [next(iter(t.keys())) for t in trajectory]
    traj_values = [next(iter(t.values())) for t in trajectory]

    return {
        "traj_keys": traj_keys,
        "traj_values": traj_values,
    }

pal_ds = Dataset.from_list(pal_demos).map(react_trajectory)
sc_ds = Dataset.from_list(sc_demos).map(react_trajectory)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [5]:
gsm8k_baselines = DatasetDict(
    {"pal": pal_ds,
     "sc": sc_ds}
)

In [6]:
gsm8k_baselines.save_to_disk("var/gsm8k_base_prompts")

Saving the dataset (0/1 shards):   0%|          | 0/8 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/8 [00:00<?, ? examples/s]

In [22]:
react = {"webact_simple3": "\nDetermine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION. \nClaim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.\nAction 1: Search[Nikolaj Coster-Waldau]\nObservation 1: Nikolaj William Coster-Waldau (born 27 July 1970) is a Danish actor and producer. He graduated from the Danish National School of Performing Arts in Copenhagen in 1993,[1] and had his breakthrough role in Denmark with the film Nightwatch (1994). He played Jaime Lannister in the HBO fantasy drama series Game of Thrones, for which he received two Primetime Emmy Award nominations for Outstanding Supporting Actor in a Drama Series.. Coster-Waldau has appeared in numerous films in his native Denmark and Scandinavia, including Headhunters (2011) and A Thousand Times Good Night (2013). In the U.S, his debut film role was in the war film Black Hawk Down (2001), playing Medal of Honor recipient Gary Gordon.[2] He then played a detective in the short-lived Fox television series New Amsterdam (2008), and appeared in the 2009 Fox television film Virtuality, originally intended as a pilot.\nAction 2: Finish[SUPPORTS]\n\nClaim: Stranger Things is set in Bloomington, Indiana.\nAction 1: Search[Stranger Things]\nObservation 1: Stranger Things is an American science fiction horror drama television series created by the Duffer Brothers. Set in the 1980s, primarily in the fictional town of Hawkins, Indiana, the series centers on a number of mysteries and supernatural events occurring around the town and their impact on an ensemble of child and adult characters. \nAction 2: Finish[REFUTES]\n\nClaim: Beautiful reached number two on the Billboard Hot 100 in 2003.?\nAction 1: Search[Beautiful]\nObservation 1: Could not find [Beautiful]. Similar: ['Beautiful', 'Beautiful, Beautiful', 'A Beautiful Mind (film)', 'Beautiful (Christina Aguilera song)', 'Life Is Beautiful'].\nAction 2: Search[Beautiful (Christina Aguilera song)]\nObservation 2: \"Beautiful\" is a song recorded by American singer Christina Aguilera for her fourth studio album, Stripped (2002).\nAction 3: Lookup[Billboard Hot 100]\nObservation 3: (Result 1 / 3) The song peaked at number two on the Billboard Hot 100 in the United States, where it was certified Gold for 500,000 units shipped.\nAction 4: Finish[NOT ENOUGH INFO]\n\n", "cotqa_simple3": "Determine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION. \nClaim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.\nThought: Nikolaj William Coster-Waldau appeared in the 2009 Fox television film Virtuality, so he has worked with the Fox Broadcasting Company.\nAnswer: SUPPORTS\n\nClaim: Stranger Things is set in Bloomington, Indiana.\nThought: Stranger Things is in the fictional town of Hawkins, Indiana, not in Bloomington, Indiana.\nAnswer:REFUTES\n\nClaim: Beautiful reached number two on the Billboard Hot 100 in 2003.?\nThought: The song peaked at number two on the Billboard Hot 100 in the United States, but not sure if it was in 2003.\nAnswer: NOT ENOUGH INFO\n", "webqa_simple3": "Determine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION. \nClaim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.\nAnswer: SUPPORTS\n\nClaim: Stranger Things is set in Bloomington, Indiana.\nAnswer:REFUTES\n\nClaim: Beautiful reached number two on the Billboard Hot 100 in 2003.?\nAnswer: NOT ENOUGH INFO\n", "webthink_simple3": "\nDetermine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION. \nClaim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.\nThought 1: I need to search Nikolaj Coster-Waldau and find if he has worked with the Fox Broadcasting Company.\nAction 1: Search[Nikolaj Coster-Waldau]\nObservation 1: Nikolaj William Coster-Waldau (born 27 July 1970) is a Danish actor and producer. He graduated from the Danish National School of Performing Arts in Copenhagen in 1993,[1] and had his breakthrough role in Denmark with the film Nightwatch (1994). He played Jaime Lannister in the HBO fantasy drama series Game of Thrones, for which he received two Primetime Emmy Award nominations for Outstanding Supporting Actor in a Drama Series.. Coster-Waldau has appeared in numerous films in his native Denmark and Scandinavia, including Headhunters (2011) and A Thousand Times Good Night (2013). In the U.S, his debut film role was in the war film Black Hawk Down (2001), playing Medal of Honor recipient Gary Gordon.[2] He then played a detective in the short-lived Fox television series New Amsterdam (2008), and appeared in the 2009 Fox television film Virtuality, originally intended as a pilot.\nThought 2: Because he \"appeared in the 2009 Fox television film Virtuality\", he should have worked with the Fox Broadcasting Company.\nAction 2: Finish[SUPPORTS]\n\nClaim: Stranger Things is set in Bloomington, Indiana.\nThought 1: I should search for Stranger Things, and see if it is set in Bloomington, Indiana.\nAction 1: Search[Stranger Things]\nObservation 1: Stranger Things is an American science fiction horror drama television series created by the Duffer Brothers. Set in the 1980s, primarily in the fictional town of Hawkins, Indiana, the series centers on a number of mysteries and supernatural events occurring around the town and their impact on an ensemble of child and adult characters. \nThought 2: The observation says that it is set in a \"fictional town of Hawkins, Indiana\", so it is not set in Bloomington.\nAction 2: Finish[REFUTES]\n\nClaim: Beautiful reached number two on the Billboard Hot 100 in 2003.?\nThought 1: I need to search the song Beautiful and find if it reached number two on the Billboard Hot 100 in 2003.\nAction 1: Search[Beautiful]\nObservation 1: Could not find [Beautiful]. Similar: ['Beautiful', 'Beautiful, Beautiful', 'A Beautiful Mind (film)', 'Beautiful (Christina Aguilera song)', 'Life Is Beautiful'].\nThought 2: From suggestions, I should search \"Beautiful (Christina Aguilera song)\" to find the song.\nAction 2: Search[Beautiful (Christina Aguilera song)]\nObservation 2: \"Beautiful\" is a song recorded by American singer Christina Aguilera for her fourth studio album, Stripped (2002).\nThought 3: It does not mention Billboard, so I need to look up \"Billboard Hot 100\" to find if it reached number two on it in 2003.\nAction 3: Lookup[Billboard Hot 100]\nObservation 3: (Result 1 / 3) The song peaked at number two on the Billboard Hot 100 in the United States, where it was certified Gold for 500,000 units shipped.\nThought 4: It only says the song peaked at number two on the Billboard Hot 100, but not if it was in 2003. I am not sure if this claim is true or not.\nAction 4: Finish[NOT ENOUGH INFO]\n\n"}

In [24]:
print(react["webthink_simple3"])


Determine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION. 
Claim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.
Thought 1: I need to search Nikolaj Coster-Waldau and find if he has worked with the Fox Broadcasting Company.
Action 1: Search[Nikolaj Coster-Waldau]
Observation 1: Nikolaj William Coster-Waldau (born 27 July 1970) is a Danish actor and producer. He graduated from the Danish National School of Performing Arts in Copenhagen in 1993,[1] and had his breakthrough role in Denmark with the film Nightwatch (1994). He played Jaime Lannister in the HBO fantasy drama series Game of Thrones, for which he received two Primetime Emmy Award nominations for Outstanding Supporting Actor in a Drama Series.. Coster-Waldau has appeared in numerous films in his native Denmark and Scandinavia, including Headhunters (2011) and A Thousand Times Good Night (2013). In the U.S, his debut film role was in the war film Black Hawk Down 

In [ ]:
- "Determine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION."

Claim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.
Thought 1: I need to search Nikolaj Coster-Waldau and find if he has worked with the Fox Broadcasting Company.
Action 1: Search[Nikolaj Coster-Waldau]
Observation 1: Nikolaj William Coster-Waldau (born 27 July 1970) is a Danish actor and producer. He graduated from the Danish National School of Performing Arts in Copenhagen in 1993,[1] and had his breakthrough role in Denmark with the film Nightwatch (1994). He played Jaime Lannister in the HBO fantasy drama series Game of Thrones, for which he received two Primetime Emmy Award nominations for Outstanding Supporting Actor in a Drama Series.. Coster-Waldau has appeared in numerous films in his native Denmark and Scandinavia, including Headhunters (2011) and A Thousand Times Good Night (2013). In the U.S, his debut film role was in the war film Black Hawk Down (2001), playing Medal of Honor recipient Gary Gordon.[2] He then played a detective in the short-lived Fox television series New Amsterdam (2008), and appeared in the 2009 Fox television film Virtuality, originally intended as a pilot.
Thought 2: Because he "appeared in the 2009 Fox television film Virtuality", he should have worked with the Fox Broadcasting Company.
Action 2: Finish[SUPPORTS]

Claim: Stranger Things is set in Bloomington, Indiana.
Thought 1: I should search for Stranger Things, and see if it is set in Bloomington, Indiana.
Action 1: Search[Stranger Things]
Observation 1: Stranger Things is an American science fiction horror drama television series created by the Duffer Brothers. Set in the 1980s, primarily in the fictional town of Hawkins, Indiana, the series centers on a number of mysteries and supernatural events occurring around the town and their impact on an ensemble of child and adult characters.
Thought 2: The observation says that it is set in a "fictional town of Hawkins, Indiana", so it is not set in Bloomington.
Action 2: Finish[REFUTES]

Claim: Beautiful reached number two on the Billboard Hot 100 in 2003.?
Thought 1: I need to search the song Beautiful and find if it reached number two on the Billboard Hot 100 in 2003.
Action 1: Search[Beautiful]
Observation 1: Could not find [Beautiful]. Similar: ['Beautiful', 'Beautiful, Beautiful', 'A Beautiful Mind (film)', 'Beautiful (Christina Aguilera song)', 'Life Is Beautiful'].
Thought 2: From suggestions, I should search "Beautiful (Christina Aguilera song)" to find the song.
Action 2: Search[Beautiful (Christina Aguilera song)]
Observation 2: "Beautiful" is a song recorded by American singer Christina Aguilera for her fourth studio album, Stripped (2002).
Thought 3: It does not mention Billboard, so I need to look up "Billboard Hot 100" to find if it reached number two on it in 2003.
Action 3: Lookup[Billboard Hot 100]
Observation 3: (Result 1 / 3) The song peaked at number two on the Billboard Hot 100 in the United States, where it was certified Gold for 500,000 units shipped.
Thought 4: It only says the song peaked at number two on the Billboard Hot 100, but not if it was in 2003. I am not sure if this claim is true or not.
Action 4: Finish[NOT ENOUGH INFO]

In [25]:
from numpy.random import default_rng

rng = default_rng()

In [32]:
rng.choice(len([1,2]), size=4, replace=True)

array([1, 0, 1, 0])